In [341]:
import numpy as np
import pandas as pd
import unicodedata
from sklearn.preprocessing import OneHotEncoder
from openpyxl import load_workbook
from openpyxl.comments import Comment

In [342]:
# Funções de conversão
def remover_acentos_e_transformar_minusculo(texto):
    # Transforma o texto em minúsculas
    texto = texto.lower()
    # Normaliza a string para remover acentos
    texto_sem_acentos = unicodedata.normalize('NFD', texto)
    texto_sem_acentos = texto_sem_acentos.encode('ascii', 'ignore').decode('utf-8')
    return texto_sem_acentos

def str_null(df: pd.DataFrame, column: str) -> pd.DataFrame:
    cols_tela = [col for col in df.columns if col.startswith(column) and col not in [f'{column}_part_1'] ]    
    for column_name in cols_tela:
        df[column_name] = df[column_name].apply(lambda x: 1 if isinstance(x, str) else 0)        
    return df

def yes_no(df: pd.DataFrame, column: str) -> pd.DataFrame:
    cols_tela = [col for col in df.columns if col.startswith(column) and col not in [f'{column}_part_1'] ]    
    for column_name in cols_tela:
        df[column_name] = df[column_name].map({'Não': 0, 'Sim': 1, ' Não': 0, ' Sim': 1}).astype('int')
    return df

def one_hot_encode_column(df: pd.DataFrame, column_names: list[str]) -> pd.DataFrame:
    """
    Realiza One-Hot Encoding em uma coluna específica de um DataFrame, 
    inserindo as colunas codificadas imediatamente após a coluna original 
    e removendo a coluna original (de forma a evitar PerformanceWarning).

    Parâmetros
    ----------
    df : pd.DataFrame
        DataFrame que será modificado (não é modificado in-place).
    column_names : List<str>
        Nomes das colunas em que o One-Hot Encoding será aplicado.

    Retorna
    -------
    pd.DataFrame
        Novo DataFrame modificado com colunas one-hot adicionadas.
    """
    for column_name in column_names:
        if column_name not in df.columns:
            raise ValueError(f"A coluna '{column_name}' não existe no DataFrame.")

        # Cria o objeto OneHotEncoder
        enc = OneHotEncoder(sparse_output=False)

        # Seleciona a coluna como DataFrame (2D)
        X = df[[column_name]]

        # Gera array 2D encodado
        encoded_array = enc.fit_transform(X)

        # Nome das colunas encodadas
        encoded_cols = enc.get_feature_names_out([column_name])

        # Converte para DataFrame, mantendo o mesmo index
        encoded_df = pd.DataFrame(encoded_array, 
                                columns=encoded_cols, 
                                index=df.index)

        # Índice da coluna original
        col_index = df.columns.get_loc(column_name)

        # "Partes" do DataFrame:
        # - left_part = tudo antes da coluna alvo
        # - pivot_col = a própria coluna alvo
        # - right_part = tudo após a coluna alvo
        left_part = df.iloc[:, :col_index]
        pivot_col = df[[column_name]]
        right_part = df.iloc[:, col_index + 1:]

        # Concatena:
        #  1) tudo à esquerda
        #  2) a coluna original (pivot)
        #  3) as colunas encodadas
        #  4) tudo à direita
        # Depois removemos a coluna original
        temp_df = pd.concat([left_part, pivot_col, encoded_df, right_part], axis=1)

        # Remover a coluna original
        df = temp_df.drop(columns=[column_name])

    return df

# Caminho para o arquivo xlsx local
file_path = './banco_dados.xlsx'
# Carregar o DataFrame
df_inicial = pd.read_excel(file_path, sheet_name='BDados')

# Realizando várias substituições em todas as colunas do DataFrame
df_replace = df_inicial.replace({
    "other;": "",   # Remove "other;"
    "Sucess;": "",  # Remove "Sucess;"
    "Sucess": ""    # Remove "Sucess"
}, regex=True)

In [343]:
# Função para dividir as colunas que têm valores separados por ponto e vírgula
def split_columns(df):
    columns_to_drop = []  # Lista para armazenar as colunas originais a serem removidas
    for col in df.columns:
        if df[col].dtype == 'object':  # Verificar se a coluna é do tipo string
            if df[col].str.contains(';').any():  # Se o valor é uma string com ";", quebra em novas colunas
                split_df = df[col].str.split(';', expand=True)
                split_df.columns = [f'{col}_part_{i+1}' for i in range(split_df.shape[1])]
                # Adicionar as novas colunas ao DataFrame
                df = pd.concat([df, split_df], axis=1)
                # Marcar a coluna original para ser removida
                columns_to_drop.append(col)
    
    # Remover as colunas originais
    df = df.drop(columns=columns_to_drop)
    return df

# Tela 27:
df_replace['Tela 27'] = (
    df_replace['Tela 27']
    .str.replace(r';{2,}', '', regex=True)  # Remove ";;"
    .str.replace(r'; ;', ';', regex=True)   # Remove "; ;" (espaço entre ;)
    .str.replace(r'; ', ';', regex=True)    # Remove espaço após ";"
)
# Tela 53:
df_replace['Tela 53'] = (
    df_replace['Tela 53']
    .str.replace(r';{2,}', '', regex=True)  # Remove ";;"
    .str.replace(r'; ;', ';', regex=True)   # Remove "; ;" (espaço entre ;)
    .str.replace(r'; ', ';', regex=True)    # Remove espaço após ";"
)
# Aplicar a função para dividir as colunas
df_split = split_columns(df_replace)

# Remover as colunas vazias
df_null = df_split.map(lambda x: np.nan if str(x).strip() == '' else x)
df_dropna = df_null.dropna(axis=1, how='all')
df = df_dropna.fillna(0)
# Dentro de sua família, você é o(a) único(a) filho(a)?
df['Tela 02_part_8'] = df['Tela 02_part_8'].replace("Sim", 0).astype('int')
# Possui filhos(as)?
df['Tela 02_part_10'] = df['Tela 02_part_10'].replace("Não", 0).astype('int')
# Possui filhos(as) menores de 6 anos?
df['Tela 02_part_11'] = df['Tela 02_part_11'].map({'Não': 0, 'Sim': 1}).astype('int')
# Tela 42:
def pontos_42(colData) -> int:
    index = 0.0
    for item in colData.split("|"):
        str = item.split(",")
        if 43.0 < float(str[0]) < 64.0 and 393.0 < float(str[1]) < 414.0: index += 1.0 #  53.799999999999955, 403.8 
        if 422.0 < float(str[0]) < 443.0 and 38.0 < float(str[1]) < 59.0: index += 1.0 # 432.79999999999995,   48.80000000000001;                
        if 28.0 < float(str[0]) < 49.0 and 201.0 < float(str[1]) < 222.0: index += 1.0 #  38.799999999999955, 211.8;                
        if 105.0 < float(str[0]) < 126.0 and 13.0 < float(str[1]) < 34.0: index += 1.0 # 115.79999999999995,   23.80000000000001;                
        if 274.0 < float(str[0]) < 295.0 and 369.0 < float(str[1]) < 390.0: index += 1.0 # 284.79999999999995,  379.8;                
        if 341.0 < float(str[0]) < 362.0 and 232.0 < float(str[1]) < 253.0: index += 1.0 # 351.79999999999995,  242.8                                                          
    return index
df['Tela 42_part_2'] = df['Tela 42_part_2'].apply(pontos_42) 
# Tela 51: # 
def pontos_51(colData) -> int:
    index = 0.0
    for item in colData.split("|"):
        str = item.split(",")
        if 196.0 < float(str[0]) < 217.0 and 579.0 < float(str[1]) < 600.0: index += 1.0 # 206.79999999999995,589.8 
        if 105.0 < float(str[0]) < 126.0 and 585.0 < float(str[1]) < 606.0: index += 1.0 # 115.79999999999995,595.8   
        if 206.0 < float(str[0]) < 227.0 and 46.0 < float(str[1]) < 67.0: index += 1.0 # 216.79999999999995,56.80000000000001
        if 376.0 < float(str[0]) < 397.0 and 480.0 < float(str[1]) < 501: index += 1.0 # 386.79999999999995,490.8
        if 293.0 < float(str[0]) < 314.0 and 668.0 < float(str[1]) < 689.0: index += 1.0 # 303.79999999999995,678.3169811320754
    return index
df['Tela 51_part_2'] = df['Tela 51_part_2'].apply(pontos_51)    
df.insert(0,"Alvo", df['Tela 03_part_2'])

df = df.drop(columns=['Tela 03_part_2', 'key','Mac Address_part_1','Mac Address_part_2','Mac Address_part_3'])
# Usando ExcelWriter para adicionar a nova aba ao arquivo Excel existente
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='XDados', index=False)

In [344]:
wb = load_workbook(file_path)
# Selecionar a planilha (tabela)
ws = wb['XDados']
# Adicionar o comentário à célula
ws['A1'].comment = Comment("Diagnóstico em relação ao seu estado de saúde mental\n", "Resia Morais")
ws['B1'].comment = Comment("Momento em que começou a responder o Questionário\n", "Resia Morais")
ws['C1'].comment = Comment("Confirmação do Usuário que está disposto a responder o Questionário\n", "Resia Morais")
ws['D1'].comment = Comment("Momento em que começou a responder a tela 2 do Questionário\n", "Resia Morais")
ws['E1'].comment = Comment("Resposta do usuário à pergunta:\n Que horas são neste instante?\n", "Resia Morais")
ws['F1'].comment = Comment("Resposta do usuário à pergunta:\n Data de hoje?\n", "Resia Morais")
ws['G1'].comment = Comment("Resposta do usuário à pergunta:\n Qual a sua Idade?\n", "Resia Morais")
ws['H1'].comment = Comment("Resposta do usuário à pergunta:\n Qual o seu gênero?\n", "Resia Morais")
ws['I1'].comment = Comment("Resposta do usuário à pergunta:\n Qual foi o sexo atribuído no seu nascimento?\n", "Resia Morais")
ws['J1'].comment = Comment("Resposta do usuário à pergunta:\n Qual a sua Cor ou Raça ?\n", "Resia Morais")
ws['K1'].comment = Comment("Resposta do usuário à pergunta:\n Quantos irmãos você tem?\n", "Resia Morais")
ws['L1'].comment = Comment("Resposta do usuário à pergunta:\n Qual o seu estado civil?\n", "Resia Morais")
ws['M1'].comment = Comment("Resposta do usuário à pergunta:\n Quantos filhos você tem?\n", "Resia Morais")
ws['N1'].comment = Comment("Resposta do usuário à pergunta:\n Possui filhos(as) menores de 6 anos?\n", "Resia Morais")
ws['O1'].comment = Comment("Resposta do usuário à pergunta:\n Qual é a Religião?\n", "Resia Morais")
ws['P1'].comment = Comment("Resposta do usuário à pergunta:\n Qual a sua Escolaridade?\n", "Resia Morais")
ws['Q1'].comment = Comment("Resposta do usuário à pergunta:\n Qual a sua Renda familiar mensal de sua casa (somatória)?\n", "Resia Morais")
ws['R1'].comment = Comment("Momento em que começou a responder a tela 3 do Questionário\n", "Resia Morais")
ws['S1'].comment = Comment("Momento em que começou a responder a tela 7 do Questionário\n", "Resia Morais")
ws['T1'].comment = Comment("Resposta do usuário à pergunta:\n O que você viu na tela anterior?\n", "Resia Morais")
ws['U1'].comment = Comment("Momento em que começou a responder a tela 10 do Questionário\n", "Resia Morais")
ws['V1'].comment = Comment("Resposta do usuário à pergunta:\n O que você viu na tela anterior?\n", "Resia Morais")
ws['W1'].comment = Comment("Momento em que começou a responder a tela 13 do Questionário\n", "Resia Morais")
ws['X1'].comment = Comment("Resposta do usuário à pergunta:\n Qual foi a sequência correta dos números apresentados na tela anterior?\n", "Resia Morais")
ws['Y1'].comment = Comment("Momento em que começou a responder a tela 15 do Questionário\n", "Resia Morais")
ws['Z1'].comment = Comment("Resposta do usuário à pergunta:\n Qual foi a sequência correta dos números apresentados na tela anterior?\n", "Resia Morais")
ws['AA1'].comment = Comment("Momento em que começou a responder a tela 17 do Questionário\n", "Resia Morais")
ws['AB1'].comment = Comment("Resposta do usuário à pergunta:\n Qual foi a sequência correta dos números apresentados na tela anterior?\n", "Resia Morais")
ws['AC1'].comment = Comment("Momento em que começou a responder a tela 19 do Questionário\n", "Resia Morais")
ws['AD1'].comment = Comment("Resposta do usuário à pergunta:\n Qual foi a sequência correta dos números apresentados na tela anterior?\n", "Resia Morais")
ws['AE1'].comment = Comment("Momento em que começou a responder a tela 21 do Questionário\n", "Resia Morais")
ws['AF1'].comment = Comment("Resposta do usuário à pergunta:\n Qual foi a sequência correta dos números apresentados na tela anterior?\n", "Resia Morais")
ws['AG1'].comment = Comment("Momento em que começou a responder a tela 23 do Questionário\n", "Resia Morais")
ws['AH1'].comment = Comment("Resposta do usuário à pergunta:\n Qual foi a sequência correta dos números apresentados na tela anterior?\n", "Resia Morais")
ws['AI1'].comment = Comment("Momento em que começou a responder a tela 25 do Questionário\n", "Resia Morais")
ws['AJ1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AK1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AL1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AM1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AN1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AO1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AP1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AQ1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AR1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AS1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AT1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AU1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AV1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AW1'].comment = Comment("O usuário deve selecione a palavra que melhor a descreve a imagem apresentada a ele.\n", "Resia Morais")
ws['AX1'].comment = Comment("Momento em que começou a responder a tela 26 do Questionário\n", "Resia Morais")
ws['AY1'].comment = Comment("Resposta do usuário à pergunta:\n Tem sentido muito medo de perder o controle ou enlouquecer?\n", "Resia Morais")
ws['AZ1'].comment = Comment("Resposta do usuário à pergunta:\n Para impedir o ganho de peso, há uso de laxantes, diuréticos ou outros medicamentos; jejum, vômitos autoinduzidos, ou exercício físico em excesso?\n", "Resia Morais")
ws['BA1'].comment = Comment("Resposta do usuário à pergunta:\n Sente medo intenso de ganhar peso ou de engordar, ao ponto de não comer?\n", "Resia Morais")
ws['BB1'].comment = Comment("Resposta do usuário à pergunta:\n Há ingestão persistente de substâncias não nutritivas tais como doces e/ou chocolates, durante um período mínimo de um mês?\n", "Resia Morais")
ws['BC1'].comment = Comment("Resposta do usuário à pergunta:\n Você se irrita fácil, de forma que seu humor muda facilmente durante o dia?\n", "Resia Morais")
ws['BD1'].comment = Comment("Resposta do usuário à pergunta:\n Já teve ataques ou crises de medo intenso nos quais se sentiu muito mal?\n", "Resia Morais")
ws['BE1'].comment = Comment("Momento em que começou a responder a tela 25 do Questionário\n", "Resia Morais")
ws['BF1'].comment = Comment("O usuário deve selecione uma imagem que completa a imagem apresentada a ele.\n", "Resia Morais")
ws['BG1'].comment = Comment("Momento em que começou a responder a tela 28 do Questionário\n", "Resia Morais")
ws['AH1'].comment = Comment("Resposta do usuário à pergunta:\n Ultimamente tem arrancado o próprio cabelo de forma recorrente, resultando em perda de cabelo?\n", "Resia Morais")
ws['AI1'].comment = Comment("Resposta do usuário à pergunta:\n Observou grande perda de interesse ou prazer em todas ou quase todas as atividades na maior parte do dia; sente-se triste, vazio, sem esperança?\n", "Resia Morais")
ws['BJ1'].comment = Comment("Resposta do usuário à pergunta:\n Você se sente incomodado por pensamentos que surgem em sua mente, mesmo quando não os deseja, como o medo de estar exposto a germes, doenças ou sujeira, ou a necessidade de que tudo esteja alinhado de certa maneira?\n", "Resia Morais")
ws['BK1'].comment = Comment("Resposta do usuário à pergunta:\n Você é incomodado por imagens indesejadas que surgem em sua mente, como cenas violentas e horríveis, ou conteúdo de natureza sexual?\n", "Resia Morais")
ws['BL1'].comment = Comment("Resposta do usuário à pergunta:\n Percebo que sou uma pessoa especial e único/a? Espero que um dia as pessoas possam reconhecer meu valor e a diferença que faço na vida delas.\n", "Resia Morais")
ws['BM1'].comment = Comment("Resposta do usuário à pergunta:\n Tende a se enxergar como alguém socialmente incapaz, sem atrativos pessoais ou inferior aos outros?\n", "Resia Morais")
ws['BN1'].comment = Comment("Resposta do usuário à pergunta:\n Possui dificuldade em iniciar projetos ou fazer as coisas sozinho (por falta de autoconfiança, em vez de falta de motivação ou energia)?\n", "Resia Morais")
ws['BO1'].comment = Comment("Momento em que começou a responder a tela 29 do Questionário\n", "Resia Morais")
ws['BP1'].comment = Comment("Usuário precisa completar a sequencia:\n 2, 4, 8, 16, ?\n", "Resia Morais")
ws['BQ1'].comment = Comment("Usuário precisa completar a sequencia:\n 1, 3, 9, ?\n", "Resia Morais")    
ws['BR1'].comment = Comment("Usuário precisa completar a sequencia:\n 3, 7, 11, 15, ?\n", "Resia Morais") 
ws['BS1'].comment = Comment("Usuário precisa completar a sequencia:\n 32, 16, 8, ? \n", "Resia Morais") 
ws['BT1'].comment = Comment("Momento em que começou a responder a tela 30 do Questionário\n", "Resia Morais")
ws['BU1'].comment = Comment("Resposta do usuário à pergunta:\n Forme uma frase que faça sentido colocando as palavras em ordem correta:\n", "Resia Morais")
ws['BV1'].comment = Comment("Momento em que começou a responder a tela 31 do Questionário\n", "Resia Morais")
ws['BW1'].comment = Comment("Resposta do usuário à pergunta:\n Sente algo estranho dentro do seu corpo, e/ou movimentos, como se levasse um empurrão?\n", "Resia Morais")
ws['BX1'].comment = Comment("Resposta do usuário à pergunta:\n Sente como se alguém lhe tocasse, beliscassem, batessem ou beijassem seu corpo?\n", "Resia Morais")
ws['BY1'].comment = Comment("Resposta do usuário à pergunta:\n Possui a sensação de falta de controle durante a alimentação; não consegue parar de comer ou controlar a quantidade que está ingerindo?\n", "Resia Morais")
ws['BZ1'].comment = Comment("Resposta do usuário à pergunta:\n Sente-se descansado e preparado para mais um dia, mesmo com apenas 3 horas de sono?\n", "Resia Morais")
ws['CA1'].comment = Comment("Resposta do usuário à pergunta:\n Você com frequência tem dificuldade em manter-se concentrado durante a realização de tarefas ou atividades?\n", "Resia Morais")
ws['CB1'].comment = Comment("Resposta do usuário à pergunta:\n Muitas vezes responde precipitadamente antes que as perguntas tenham sido completadas.\n", "Resia Morais")
ws['CC1'].comment = Comment("Momento em que começou a responder a tela 32 do Questionário\n", "Resia Morais")
ws['CD1'].comment = Comment("Resposta do usuário à pergunta:\n Cidade onde você está agora?\n", "Resia Morais")
ws['CE1'].comment = Comment("Resposta do usuário à pergunta:\n Estado onde você está agora?\n", "Resia Morais")
ws['CF1'].comment = Comment("Momento em que começou a responder a tela 33 do Questionário\n", "Resia Morais")
ws['CG1'].comment = Comment("Resposta do usuário à pergunta:\n Selecione qual o período do dia atual?\n", "Resia Morais")
ws['CH1'].comment = Comment("Momento em que começou a responder a tela 41 do Questionário\n", "Resia Morais")
ws['CI1'].comment = Comment("Resposta do usuário à pergunta:\n Diariamente, há momentos em que você sente como se o chão oscilasse?\n", "Resia Morais")
ws['CJ1'].comment = Comment("Resposta do usuário à pergunta:\n Durante o dia há vários momentos em que você se sente irritado(a) e impaciente?\n", "Resia Morais")
ws['CK1'].comment = Comment("Resposta do usuário à pergunta:\n Diariamente, há momentos em que você se sente muito feliz, irradiante?\n", "Resia Morais")
ws['CL1'].comment = Comment("Resposta do usuário à pergunta:\n Durante o dia há vários momentos em que você se sente triste e/ou melancólico(a)?\n", "Resia Morais")
ws['CM1'].comment = Comment("Resposta do usuário à pergunta:\n Tem notado(a) muita preocupação com um ou mais defeitos ou falhas percebidas na aparência física que não são observáveis ou que parecem leves para os outros.\n", "Resia Morais")
ws['CN1'].comment = Comment("Resposta do usuário à pergunta:\n Você com frequência mexe de forma irrequieta as mãos e os pés ou remexe-se na cadeira quando está sentado(a)?\n", "Resia Morais")
ws['CO1'].comment = Comment("Resposta do usuário à pergunta:\n Exibe dificuldade em concordar com as ideias de outras pessoas, demonstrando inflexibilidade e rigidez tanto em relação a si mesmo quanto aos outros.\n", "Resia Morais")
ws['CP1'].comment = Comment("Momento em que começou a responder a tela 42 do Questionário\n", "Resia Morais")
ws['CQ1'].comment = Comment("Resposta do usuário à pergunta:\n Encontre as 6 imagens que apareceção anteriormente aqui?\n", "Resia Morais")
ws['CR1'].comment = Comment("Momento em que começou a responder a tela 43 do Questionário\n", "Resia Morais")
ws['CS1'].comment = Comment("Resposta do usuário à pergunta:\n Encontrar-se com um ente querido falecido.\n", "Resia Morais")
ws['CT1'].comment = Comment("Resposta do usuário à pergunta:\n Ver uma pessoa nua\n", "Resia Morais")
ws['CU1'].comment = Comment("Resposta do usuário à pergunta:\n Saber como desempenhar tarefas cotidianas sem ser prejudicado por pensamentos incessantes que me limitam.\n", "Resia Morais")
ws['CV1'].comment = Comment("Resposta do usuário à pergunta:\n Voltar na minha infância e recomeçar tudo\n", "Resia Morais")
ws['CW1'].comment = Comment("Resposta do usuário à pergunta:\n Voltar na minha adolescência e recomeçar tudo\n", "Resia Morais")
ws['CX1'].comment = Comment("Resposta do usuário à pergunta:\n Saber se meu namorado (a) ou esposo (a) está me traindo\n", "Resia Morais")
ws['CY1'].comment = Comment("Resposta do usuário à pergunta:\n Saber como poderia ser o futuro da minha família e ajudá-los\n", "Resia Morais")
ws['CZ1'].comment = Comment("Resposta do usuário à pergunta:\n Desaparecer no tempo e espaço, ao entrar pela fenda\n", "Resia Morais")
ws['DA1'].comment = Comment("Resposta do usuário à pergunta:\n Saber se estarei vivo daqui 5 anos.\n", "Resia Morais")
ws['DB1'].comment = Comment("Resposta do usuário à pergunta:\n Saber quem é minha alma gêmea.\n", "Resia Morais")
ws['DC1'].comment = Comment("Resposta do usuário à pergunta:\n Ver meu futuro profissional.\n", "Resia Morais")
ws['DD1'].comment = Comment("Resposta do usuário à pergunta:\n Testemunhar avanços tecnológicos futuros.\n", "Resia Morais")
ws['DE1'].comment = Comment("Resposta do usuário à pergunta:\n Presenciar um evento histórico de grande relevância, como a construção das pirâmides do Egito, a Grande Muralha da China, a era dos dinossauros, ou a criação de alguma das invenções de Albert Einstein ou Leonardo da Vinci, por exemplo.\n", "Resia Morais")
ws['DF1'].comment = Comment("Resposta do usuário à pergunta:\n Saber se ficarei rico(a)\n", "Resia Morais")
ws['DG1'].comment = Comment("Resposta do usuário à pergunta:\n Saber como faço para dormir a noite toda.\n", "Resia Morais")
ws['DH1'].comment = Comment("Resposta do usuário à pergunta:\n Saber como fazer meu cônjuge e/ou filho(a) a realizar tarefas e tomar decisões em suas vidas de acordo com meus valores e princípios.\n", "Resia Morais")
ws['DI1'].comment = Comment("Resposta do usuário à pergunta:\n Saber como desaparecer minha dor e sofrimento existencial.\n", "Resia Morais")
ws['DJ1'].comment = Comment("Momento em que começou a responder a tela 44 do Questionário\n", "Resia Morais")
ws['DK1'].comment = Comment("Resposta do usuário à pergunta:\n Percebe-se impulsivo(a) e de forma abusiva; p. ex., gastos exagerados, sexo com desconhecidos, abuso de substâncias, dirigir de forma perigosa/irresponsável e/ou atos repetitivos de autolesão?\n", "Resia Morais")
ws['DL1'].comment = Comment("Resposta do usuário à pergunta:\n Tem dificuldade em terminar o que começa?\n", "Resia Morais")
ws['DM1'].comment = Comment("Resposta do usuário à pergunta:\n Quando você conta uma história que aconteceu, na maioria das vezes você exagera e dramatiza alguns detalhes?\n", "Resia Morais")
ws['DN1'].comment = Comment("Resposta do usuário à pergunta:\n Já testemunhou e/ou ainda é exposto(a) de forma repetida ou extrema a detalhes aversivos do evento traumático?\n", "Resia Morais")
ws['DO1'].comment = Comment("Resposta do usuário à pergunta:\n Você apresenta explosões de raiva recorrente, manifestadas por meio da linguagem (como insultos dirigidos a outras pessoas) ou através de ações (como agressão física)?\n", "Resia Morais")
ws['DP1'].comment = Comment("Resposta do usuário à pergunta:\n Você está tendo insônia praticamente todos os dias?\n", "Resia Morais")
ws['DQ1'].comment = Comment("Momento em que começou a responder a tela 47 do Questionário\n", "Resia Morais")
ws['DR1'].comment = Comment("Resposta do usuário à pergunta:\n Qual das opções corresponde ao som escutado?\n", "Resia Morais")        
ws['DS1'].comment = Comment("Momento em que começou a responder a tela 48 do Questionário\n", "Resia Morais")
ws['DT1'].comment = Comment("Resposta do usuário à pergunta:\n Selecione qual alternativa corresponde ao que você vê na imagem acima.\n", "Resia Morais")            
ws['DU1'].comment = Comment("Momento em que começou a responder a tela 49 do Questionário\n", "Resia Morais")
ws['DV1'].comment = Comment("Resposta do usuário à pergunta:\n O que você deseja desenhar?\n", "Resia Morais")  
ws['DW1'].comment = Comment("Resposta do usuário à pergunta:\n O desenho:\n", "Resia Morais")           
ws['DX1'].comment = Comment("Momento em que começou a responder a tela 50 do Questionário\n", "Resia Morais")
ws['DY1'].comment = Comment("Resposta do usuário à pergunta:\n Você tem sentido mais fadiga ou perda de energia quase todos os dias?\n", "Resia Morais")
ws['DZ1'].comment = Comment("Resposta do usuário à pergunta:\n Você vivenciou algum evento traumático?\n", "Resia Morais")
ws['EA1'].comment = Comment("Resposta do usuário à pergunta:\n Você tem escutado Zumbido no ouvido?\n", "Resia Morais")
ws['EB1'].comment = Comment("Resposta do usuário à pergunta:\n Você tem dificuldades em jogar fora objetos usados ou sem valor, mesmo quando não têm valor sentimental. Ultimamente guarda muitas coisas, papeis, recibos, com a ideia de que poderá precisar algum dia?\n", "Resia Morais")
ws['EC1'].comment = Comment("Resposta do usuário à pergunta:\n Você apresenta comportamentos repetitivos (p. ex., lavar as mãos, organizar, verificar) ou atos mentais (p. ex., orar, contar ou repetir palavras em silêncio).\n", "Resia Morais")
ws['ED1'].comment = Comment("Resposta do usuário à pergunta:\n Você se sente incomodado por impulsos como machucar alguém que você ama, mesmo quando não deseja fazê-lo?\n", "Resia Morais")
ws['EE1'].comment = Comment("Resposta do usuário à pergunta:\n Você tem visto algo estranho como figuras, sombras, fogo, fantasmas, demônios, pessoas estranhas ou algo do tipo, no seu dia a dia?\n", "Resia Morais")
ws['EF1'].comment = Comment("Momento em que começou a responder a tela 51 do Questionário\n", "Resia Morais")
ws['EG1'].comment = Comment("Resposta do usuário à pergunta:\n Encontre os 5 erros?\n", "Resia Morais")  
ws['EH1'].comment = Comment("Momento em que começou a responder a tela 52 do Questionário\n", "Resia Morais")
ws['EI1'].comment = Comment("Resposta do usuário à pergunta:\n Nos últimos meses, você tem ouvido vozes de pessoas estranhas?\n", "Resia Morais")
ws['EJ1'].comment = Comment("Resposta do usuário à pergunta:\n Ultimamente, você tem ouvido vozes que são seus próprios pensamentos sendo expressos em voz alta?\n", "Resia Morais")
ws['EK1'].comment = Comment("Resposta do usuário à pergunta:\n Alguém tem tentado envenená-lo(a)?\n", "Resia Morais")
ws['EL1'].comment = Comment("Resposta do usuário à pergunta:\n Você tem dificuldade para relaxar? Está sempre ocupado(a)?\n", "Resia Morais")
ws['EM1'].comment = Comment("Resposta do usuário à pergunta:\n Você vem apresentando sensações de falta de ar ou sufocamento?\n", "Resia Morais")
ws['EN1'].comment = Comment("Resposta do usuário à pergunta:\n Você está vivenciando um estado de luto prolongado e persistente, que se estende por um período superior a 12 meses, caracterizado por uma intensa saudade, preocupação e apatia em relação ao futuro?\n", "Resia Morais")
ws['EO1'].comment = Comment("Resposta do usuário à pergunta:\n Você tem medo de ser demitido(a) o tempo todo?\n", "Resia Morais")     
ws['EP1'].comment = Comment("Momento em que começou a responder a tela 53 do Questionário\n", "Resia Morais")
ws['EQ1'].comment = Comment("Resposta do usuário à pergunta:\n Qual das imagens abaixo completa a sequência a seguir?\n", "Resia Morais") 
ws['ER1'].comment = Comment("Momento em que começou a responder a tela 54 do Questionário\n", "Resia Morais")
ws['ES1'].comment = Comment("Resposta do usuário à pergunta:\n Você vive com medo de decepcionar as pessoas?\n", "Resia Morais")
ws['ET1'].comment = Comment("Resposta do usuário à pergunta:\n Você tem-se irritado com mais facilidade que antes?\n", "Resia Morais")
ws['EU1'].comment = Comment("Resposta do usuário à pergunta:\n Nos últimos meses você sente dificuldades de parar de se preocupar?\n", "Resia Morais")
ws['EV1'].comment = Comment("Resposta do usuário à pergunta:\n Você pensa muitas coisas ao mesmo tempo?\n", "Resia Morais")
ws['EW1'].comment = Comment("Resposta do usuário à pergunta:\n Você sente dificuldade em se concentrar?\n", "Resia Morais")
ws['EX1'].comment = Comment("Resposta do usuário à pergunta:\n Evita realizar atividades profissionais ou estudantis que impliquem contato interpessoal, pois tem muito medo de críticas, desaprovação ou rejeição?\n", "Resia Morais")
ws['EY1'].comment = Comment("Resposta do usuário à pergunta:\n Você não acha que tem fraquezas e muito menos evita ambientes, pois consegue tudo o que quer?\n", "Resia Morais")    
ws['EZ1'].comment = Comment("Momento em que começou a responder a tela 57 do Questionário\n", "Resia Morais")
ws['FA1'].comment = Comment("Resposta do usuário à pergunta:\n coquetel\n", "Resia Morais") 
ws['FB1'].comment = Comment("Resposta do usuário à pergunta:\n humburgue\n", "Resia Morais")
ws['FC1'].comment = Comment("Resposta do usuário à pergunta:\n casa\n", "Resia Morais")
ws['FD1'].comment = Comment("Resposta do usuário à pergunta:\n gato\n", "Resia Morais")
ws['FE1'].comment = Comment("Resposta do usuário à pergunta:\n trabalhador\n", "Resia Morais")
ws['FF1'].comment = Comment("Resposta do usuário à pergunta:\n carro\n", "Resia Morais")
ws['FG1'].comment = Comment("Resposta do usuário à pergunta:\n cachorro\n", "Resia Morais")
ws['FH1'].comment = Comment("Resposta do usuário à pergunta:\n passaro\n", "Resia Morais")
ws['FI1'].comment = Comment("Resposta do usuário à pergunta:\n cerveja\n", "Resia Morais")
ws['FJ1'].comment = Comment("Resposta do usuário à pergunta:\n cocacola\n", "Resia Morais")
ws['FK1'].comment = Comment("Resposta do usuário à pergunta:\n cafe\n", "Resia Morais")
ws['FL1'].comment = Comment("Resposta do usuário à pergunta:\n bombons\n", "Resia Morais")
ws['FM1'].comment = Comment("Resposta do usuário à pergunta:\n viajar\n", "Resia Morais")
ws['FN1'].comment = Comment("Resposta do usuário à pergunta:\n livros\n", "Resia Morais")
ws['FO1'].comment = Comment("Resposta do usuário à pergunta:\n leao\n", "Resia Morais")
ws['FP1'].comment = Comment("Resposta do usuário à pergunta:\n controlegame\n", "Resia Morais")
ws['FQ1'].comment = Comment("Momento em que começou a responder a tela 58 do Questionário\n", "Resia Morais")
ws['FR1'].comment = Comment("Resposta do usuário à pergunta:\n Você acredita que precisa fazer as coisas de forma perfeita, caso contrário não será aceito (a)?\n", "Resia Morais")
ws['FS1'].comment = Comment("Resposta do usuário à pergunta:\n Você acredita que as pessoas estão julgando suas atitudes e comportamentos, mesmo que não estejam dizendo?\n", "Resia Morais")
ws['FT1'].comment = Comment("Resposta do usuário à pergunta:\n Você tem ideias negativas, como pensar em morrer?\n", "Resia Morais")
ws['FU1'].comment = Comment("Resposta do usuário à pergunta:\n Você se sente inadequado (a) inquieto (a) e conversa de forma excessiva?\n", "Resia Morais")
ws['FV1'].comment = Comment("Resposta do usuário à pergunta:\n Você costuma adiar ou evitar fazer as coisas até o último minuto? Possui o perfil de procrastinar?\n", "Resia Morais")
ws['FW1'].comment = Comment("Resposta do usuário à pergunta:\n Tem relutado em delegar tarefas ou de trabalhar com outras pessoas, pois precisa estar no controle de tudo.\n", "Resia Morais")
ws['FX1'].comment = Comment("Resposta do usuário à pergunta:\n Você percebe que se esforça de forma intensa para evitar ser abandonado (a)?\n", "Resia Morais") 
ws['FY1'].comment = Comment("Momento em que começou a responder a tela 59 do Questionário\n", "Resia Morais")
ws['FZ1'].comment = Comment("Resposta do usuário à pergunta:\n Qual das opções corresponde ao som escutado?\n", "Resia Morais") 
ws['GA1'].comment = Comment("Momento em que começou a responder a tela 60 do Questionário\n", "Resia Morais")
ws['GB1'].comment = Comment("Resposta do usuário à pergunta:\n Qual a emoção básica da imagem:\n", "Resia Morais") 
ws['GC1'].comment = Comment("Resposta do usuário à pergunta:\n Qual a emoção básica da imagem:\n", "Resia Morais")
ws['GD1'].comment = Comment("Resposta do usuário à pergunta:\n Qual a emoção básica da imagem:\n", "Resia Morais")
ws['GE1'].comment = Comment("Resposta do usuário à pergunta:\n Qual a emoção básica da imagem:\n", "Resia Morais")
ws['GF1'].comment = Comment("Resposta do usuário à pergunta:\n Qual a emoção básica da imagem:\n", "Resia Morais")
ws['GG1'].comment = Comment("Resposta do usuário à pergunta:\n Qual a emoção básica da imagem:\n", "Resia Morais")
ws['GH1'].comment = Comment("Momento em que começou a responder a tela 62 do Questionário\n", "Resia Morais")
ws['GI1'].comment = Comment("Quando a resposta é o emoji:\n sempre_atrasado\n", "Resia Morais")
ws['GJ1'].comment = Comment("Quando a resposta é o emoji:\n poderoso\n", "Resia Morais") 
ws['GK1'].comment = Comment("Quando a resposta é o emoji:\n otimista\n", "Resia Morais") 
ws['GL1'].comment = Comment("Quando a resposta é o emoji:\n em_panico\n", "Resia Morais") 
ws['GM1'].comment = Comment("Quando a resposta é o emoji:\n indeciso\n", "Resia Morais") 
ws['GN1'].comment = Comment("Quando a resposta é o emoji:\n triste\n", "Resia Morais") 
ws['GO1'].comment = Comment("Quando a resposta é o emoji:\n entediado\n", "Resia Morais") 
ws['GP1'].comment = Comment("Quando a resposta é o emoji:\n pessimista\n", "Resia Morais") 
ws['GQ1'].comment = Comment("Quando a resposta é o emoji:\n reflexivo\n", "Resia Morais") 
ws['GR1'].comment = Comment("Quando a resposta é o emoji:\n confiante\n", "Resia Morais") 
ws['GS1'].comment = Comment("Quando a resposta é o emoji:\n forte\n", "Resia Morais") 
ws['GT1'].comment = Comment("Quando a resposta é o emoji:\n nojo\n", "Resia Morais") 
ws['GU1'].comment = Comment("Momento em que começou a responder a tela 63 do Questionário\n", "Resia Morais")
ws['GV1'].comment = Comment("Quando a resposta é o emoji:\n esperancoso\n", "Resia Morais") 
ws['GW1'].comment = Comment("Quando a resposta é o emoji:\n pura_alegria\n", "Resia Morais")
ws['GX1'].comment = Comment("Quando a resposta é o emoji:\n burro\n", "Resia Morais") 
ws['GY1'].comment = Comment("Quando a resposta é o emoji:\n surpreso\n", "Resia Morais") 
ws['GZ1'].comment = Comment("Quando a resposta é o emoji:\n feliz\n", "Resia Morais") 
ws['HA1'].comment = Comment("Quando a resposta é o emoji:\n velho\n", "Resia Morais") 
ws['HB1'].comment = Comment("Quando a resposta é o emoji:\n frustrado\n", "Resia Morais") 
ws['HC1'].comment = Comment("Quando a resposta é o emoji:\n ansioso\n", "Resia Morais") 
ws['HD1'].comment = Comment("Quando a resposta é o emoji:\n emocionado\n", "Resia Morais") 
ws['HE1'].comment = Comment("Quando a resposta é o emoji:\n em_paz\n", "Resia Morais") 
ws['HF1'].comment = Comment("Quando a resposta é o emoji:\n inteligente\n", "Resia Morais") 
ws['HG1'].comment = Comment("Quando a resposta é o emoji:\n preocupado\n", "Resia Morais") 
ws['HH1'].comment = Comment("Momento em que começou a responder a tela 64 do Questionário\n", "Resia Morais")
ws['HI1'].comment = Comment("Quando a resposta é o emoji:\n apaixonado\n", "Resia Morais") 
ws['HJ1'].comment = Comment("Quando a resposta é o emoji:\n desesperado\n", "Resia Morais")
ws['HK1'].comment = Comment("Quando a resposta é o emoji:\n envergonhado\n", "Resia Morais") 
ws['HL1'].comment = Comment("Quando a resposta é o emoji:\n abencoado\n", "Resia Morais") 
ws['HM1'].comment = Comment("Quando a resposta é o emoji:\n impulsivo\n", "Resia Morais") 
ws['HN1'].comment = Comment("Quando a resposta é o emoji:\n amado\n", "Resia Morais") 
ws['HO1'].comment = Comment("Quando a resposta é o emoji:\n confuso\n", "Resia Morais") 
ws['HP1'].comment = Comment("Quando a resposta é o emoji:\n sem_forcas\n", "Resia Morais") 
ws['HQ1'].comment = Comment("Quando a resposta é o emoji:\n sonolento\n", "Resia Morais") 
ws['HR1'].comment = Comment("Quando a resposta é o emoji:\n gordo\n", "Resia Morais") 
ws['HS1'].comment = Comment("Quando a resposta é o emoji:\n sem_paciencia\n", "Resia Morais") 
ws['HT1'].comment = Comment("Quando a resposta é o emoji:\n com_ciumes\n", "Resia Morais")
ws['HU1'].comment = Comment("Momento em que começou a responder a tela 65 do Questionário\n", "Resia Morais")
ws['HV1'].comment = Comment("Quando a resposta é o emoji:\n com_raiva\n", "Resia Morais") 
ws['HW1'].comment = Comment("Quando a resposta é o emoji:\n silenciado\n", "Resia Morais")
ws['HX1'].comment = Comment("Quando a resposta é o emoji:\n desanimado\n", "Resia Morais") 
ws['HY1'].comment = Comment("Quando a resposta é o emoji:\n com_gratidao\n", "Resia Morais") 
ws['HZ1'].comment = Comment("Quando a resposta é o emoji:\n fraude\n", "Resia Morais") 
ws['IA1'].comment = Comment("Quando a resposta é o emoji:\n agressivo\n", "Resia Morais") 
ws['IB1'].comment = Comment("Quando a resposta é o emoji:\n desequilibrado\n", "Resia Morais") 
ws['IC1'].comment = Comment("Quando a resposta é o emoji:\n comendo_muito\n", "Resia Morais") 
ws['ID1'].comment = Comment("Quando a resposta é o emoji:\n fumando_muito\n", "Resia Morais") 
ws['IE1'].comment = Comment("Quando a resposta é o emoji:\n jesus\n", "Resia Morais") 
ws['IF1'].comment = Comment("Quando a resposta é o emoji:\n bebendo_muito\n", "Resia Morais") 
ws['IG1'].comment = Comment("Quando a resposta é o emoji:\n animado\n", "Resia Morais")
ws['IH1'].comment = Comment("Momento em que começou a responder a tela 66 do Questionário\n", "Resia Morais")
ws['IJ1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IK1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais")
ws['IL1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IM1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IN1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IO1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IP1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IQ1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IR1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IS1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IT1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IU1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais")
ws['IV1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IW1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IX1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IY1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IZ1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JA1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JB1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JC1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JD1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JE1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais")
ws['JF1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JG1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JH1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JI1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JJ1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JK1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JL1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JM1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JN1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JO1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais")
ws['JP1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JQ1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JL1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['JM1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IP1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IQ1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IR1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IS1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IT1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IU1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais")
ws['IL1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IM1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IN1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IO1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IP1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IQ1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IR1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IS1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IT1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IU1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais")
ws['IL1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IM1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IN1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IO1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IP1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IQ1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IR1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IS1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IT1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IU1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais")
ws['IL1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IM1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IN1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IO1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IP1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IH1'].comment = Comment("Momento em que começou a responder a tela 69 do Questionário\n", "Resia Morais")
ws['IL1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IM1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IN1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IO1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IP1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IQ1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IR1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IS1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IT1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais") 
ws['IU1'].comment = Comment("Quando a resposta é o emoji:\n \n", "Resia Morais")
ws['LI1'].comment = Comment("Momento em que começou a responder a tela 70 do Questionário\n", "Resia Morais")
ws['LJ1'].comment = Comment("Resposta do usuário à pergunta:\n Você se preocupa com a internet (pensa sobre atividades virtuais anteriores ou fica antecipando quando ocorrerá a próxima conexão)?\n", "Resia Morais")
ws['LK1'].comment = Comment("Resposta do usuário à pergunta:\n Você sente necessidade de usar a internet por períodos cada vez maiores para se sentir satisfeito?\n", "Resia Morais")
ws['LL1'].comment = Comment("Resposta do usuário à pergunta:\n Você já se esforçou repetidas vezes para controlar, diminuir ou parar de usar a internet, mas fracassou?\n", "Resia Morais")
ws['LM1'].comment = Comment("Resposta do usuário à pergunta:\n Você fica inquieto, mal-humorado, deprimido ou irritável quando tenta diminuir, parar de usar a internet ou quando o uso é restringido?\n", "Resia Morais")
ws['LN1'].comment = Comment("Resposta do usuário à pergunta:\n Você fica online mais tempo do que pretendia originalmente?\n", "Resia Morais")
ws['LO1'].comment = Comment("Resposta do usuário à pergunta:\n Você já prejudicou ou correu o risco de perder um relacionamento significativo, emprego ou oportunidade educacional ou profissional por causa de internet?\n", "Resia Morais")
ws['LP1'].comment = Comment("Resposta do usuário à pergunta:\n Você já mentiu para familiares, terapeutas ou outras pessoas para esconder a extensão do seu envolvimento com a internet?\n", "Resia Morais")
ws['LQ1'].comment = Comment("Resposta do usuário à pergunta:\n Você usa a internet como uma maneira de fugir de problemas ou de aliviar um humor desagradável (por exemplo, sentimento de impotência, solidão, culpa, tristeza, ansiedade, depressão)?\n", "Resia Morais")

ws['LX1'].comment = Comment("Momento em que começou a responder a tela 72 do Questionário\n", "Resia Morais")
ws['LY1'].comment = Comment("Resposta do usuário à pergunta:\n Você tem medo ou ansiedade acentuados acerca de um objeto ou situação (p. ex., voar, altura, animais, tomar uma injeção, ver sangue).\n", "Resia Morais")
ws['LZ1'].comment = Comment("Resposta do usuário à pergunta:\n Você apresenta dificuldade persistente de descartar ou de se desfazer de pertences, independentemente do seu valor real.\n", "Resia Morais")
ws['LA1'].comment = Comment("Resposta do usuário à pergunta:\n Possui medo ou ansiedade acentuada acerca de uma ou mais situações sociais em que é exposto a uma possível avaliação por outras pessoas. Exemplos incluem interações sociais (p. ex., manter uma conversa, encontrar pessoas que não são familiares), ser observado (p. ex., comendo ou bebendo) e situações de desempenho diante de outros (p. ex., proferir palestras, falar em público).\n", "Resia Morais")
ws['LB1'].comment = Comment("Resposta do usuário à pergunta:\n Você não tem amigos íntimos ou confidentes que não sejam membros da família de primeiro grau. Geralmente, busca atividades de lazer e/ou profissionais de forma solitária?\n", "Resia Morais")
ws['LC1'].comment = Comment("Resposta do usuário à pergunta:\n Você adota um estilo bem baixo de gastos em relação a si e a outros; enxergando o dinheiro como algo a ser poupado para possíveis emergências futuras.\n", "Resia Morais")
ws['LD1'].comment = Comment("Resposta do usuário à pergunta:\n Você vem tendo sentimentos de inutilidade ou culpa excessiva?\n", "Resia Morais")
ws['LE1'].comment = Comment("Resposta do usuário à pergunta:\n Você iniciou uma quantidade maior de projetos do que o normal ou se envolveu em atividades mais arriscadas do que o habitual?\n", "Resia Morais")

# Congela a linha superior e a primeira coluna
ws.freeze_panes = "B2"

# Salvar o arquivo
wb.save(file_path)

In [345]:
# Convertendo para datetime com formato brasileiro
def extract_timestamp(date_str): 
    return pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S.%f')
# Gerar a lista de colunas para aplicar a função
columns_to_apply  = [f'Tela {i+1:02}_part_1' for i in range(77)]
# Filtrar as colunas que realmente existem no DataFrame
existing_columns = [col for col in columns_to_apply if col in df.columns]
# Aplicar a função apenas nas colunas existentes
df[existing_columns] = df[existing_columns].map(extract_timestamp)

timeInicial = df['Tela 01_part_1']
# Subtrair a próxima coluna pela coluna anterior
for i in range(1, len(existing_columns)):
    df[existing_columns[i-1]] = (df[existing_columns[i]] - df[existing_columns[i-1]]).dt.total_seconds()
df['Tela 77_part_1'] = (df['Tela 77_part_1'] - timeInicial).dt.total_seconds() / 60

df["Alvo"] = df["Alvo"].map({             
        " Não": 0.0,
        " Transtorno do Espectro Autista": 1.0/1024,
        " Transtornos Depressivos": 2.0/1024,
        " Transtorno Ciclotímico": 4.0/1024,
        " Transtornos de Ansiedade": 8.0/1024,
        " Transtorno de Estresse Pós-traumático": 16.0/1024,
        " Transtornos Alimentares": 32.0/1024,
        " Transtorno Bipolar": 64.0/1024,
        " Transtorno Obsessivo-compulsivo": 128.0/1024,
        " Transtorno de Déficit de Atenção/Hiperatividade": 256.0/1024,
        " Transtorno da Personalidade Borderline": 512.0/1024,
        " Transtorno do Espectro da Esquizofrenia e Outros Transtornos Psicóticos": 1024.0/1024, 
        " Trans. Depressivo + Trans. Ansiedade" : (2.0 + 8.0)/1024 
    }).astype('float')
# Calcula o quanto o entrevistado estava errado em noção do dia em que ele esta
df['Tela 02_part_2'] = pd.to_datetime(df['Tela 02_part_3'] + df['Tela 02_part_2'],dayfirst=True, errors="coerce").sub(timeInicial).dt.total_seconds()
df = one_hot_encode_column(df,[
    'Tela 02_part_5',  # Gênero
    'Tela 02_part_6',  # Sexo do nascimento 
    'Tela 02_part_7',  # Cor ou Raça
    'Tela 02_part_9',  # Estado civil
    'Tela 02_part_12', # Religião
    'Tela 02_part_13', # Escolaridade
    'Tela 02_part_14', # Renda familiar
    ]) 

df = one_hot_encode_column(df,[
    'Tela 07_part_2', # Tela 07 -                  
    'Tela 10_part_2', # Tela 10 -         
    'Tela 13_part_2', # Tela 13 -
    'Tela 15_part_2', # Tela 15 -
    'Tela 17_part_2', # Tela 17 -
    'Tela 19_part_2', # Tela 19 -
    'Tela 21_part_2', # Tela 21 -
    'Tela 23_part_2', # Tela 23 -
    ])
# Tela 25: # Dentre as quatro alternativas de cada imagem. Selecione a palavra que melhor a descreve
cols_tela = [col for col in df.columns if col.startswith('Tela 25_') and col not in ['Tela 25_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# Tela 26: # Responda as questões abaixo:
df = yes_no(df,'Tela 26')
# Tela 27: # Qual das imagens abaixo completa a sequência a seguir?
cols_tela = [col for col in df.columns if col.startswith('Tela 27_') and col not in ['Tela 27_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# Tela 28: # Responda as questões abaixo:
df = yes_no(df,'Tela 28')
# Tela 30: # Observe as palavras a seguir:
cols_tela = [col for col in df.columns if col.startswith('Tela 30_') and col not in ['Tela 30_part_1'] ]
df = one_hot_encode_column(df,cols_tela)          
# Tela 31: # Responda as questões abaixo: + 6
df = yes_no(df,'Tela 31')
# Tela 33: # Hora 
cols_tela = [col for col in df.columns if col.startswith('Tela 33_') and col not in ['Tela 33_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# Tela 41:
df = yes_no(df,'Tela 41')                                                                                
# Tela 43:
df = str_null(df,'Tela 43')
# Tela 44: # + 6
df = yes_no(df,'Tela 44')
# Tela 47:
cols_tela = [col for col in df.columns if col.startswith('Tela 47_') and col not in ['Tela 47_part_1'] ]
df = one_hot_encode_column(df,cols_tela)    
# Tela 48:
cols_tela = [col for col in df.columns if col.startswith('Tela 48_') and col not in ['Tela 48_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# Tela 49:
df = one_hot_encode_column(df,['Tela 49_part_2']) 
# FUTURO df['Tela 49_part_3'] = df['Tela 49_part_3'] # Quando fazer aqui remove-lo da lista drop
# Tela 50: # + 7
df = yes_no(df,'Tela 50')            
# Tela 52: # + 7
df = yes_no(df,'Tela 52')
# Tela 53: # Qual das imagens abaixo completa a sequência a seguir?
df = str_null(df,'Tela 53')                    
# Tela 54: # + 7
df = yes_no(df,'Tela 54')  
# Tela 57: # Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
df = str_null(df,'Tela 57')              
# Tela 58: # + 7
df = yes_no(df,'Tela 58') 
# Tela 59: # Qual das opções corresponde ao som escutado?
cols_tela = [col for col in df.columns if col.startswith('Tela 59_') and col not in ['Tela 59_part_1'] ]
df = one_hot_encode_column(df,cols_tela)  
# Tela 60: # + 6 Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
cols_tela = [col for col in df.columns if col.startswith('Tela 60_') and col not in ['Tela 60_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# Tela 62: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
df = str_null(df,'Tela 62')
# Tela 63: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
df = str_null(df,'Tela 63')
# Tela 64: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
df = str_null(df,'Tela 64')
# Tela 65: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
df = str_null(df,'Tela 65')
# Tela 66: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
df = str_null(df,'Tela 66')              
# Tela 69: # + 10
cols_tela = [col for col in df.columns if col.startswith('Tela 69_') and col not in ['Tela 69_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# Tela 70: # + 8
df = yes_no(df,'Tela 70')
# Tela 71: # + 5
cols_tela = [col for col in df.columns if col.startswith('Tela 71_') and col not in ['Tela 71_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# Tela 72: # + 7
df = yes_no(df,'Tela 72')
# Tela 73: # Daltonismo é o termo usado para denominar a falta de sensibilidade
cols_tela = [col for col in df.columns if col.startswith('Tela 73_') and col not in ['Tela 73_part_1'] ]
df = one_hot_encode_column(df,cols_tela)          
# Tela 74: # 
# FUTURO: Quando fazer aqui remove-lo da lista drop
# Tela 75: # 
# FUTURO: Quando fazer aqui remove-lo da lista drop
# Tela 76:
day_of_week = timeInicial.dt.day_name(locale='pt_BR').apply(remover_acentos_e_transformar_minusculo)
df['Tela 76_part_2'] = df['Tela 76_part_2'].str.strip().apply(remover_acentos_e_transformar_minusculo)
df['Tela 76_part_2'] = (df['Tela 76_part_2'] == day_of_week).astype(int)  
# Tela 77: Calcula o quanto o entrevistado estava errado em noção do tempo gasto para resolver o questionário.
def classify_time(faixa: str, valor: int) -> int: # Criar a coluna de classificação
    if faixa == ' Mais que 1 hora' and valor > 70: return 0
    elif faixa == ' 5 minutos': 
        return 5 - valor
    elif faixa == ' 15 minutos': 
        return 15 - valor
    elif faixa == ' 30 minutos': 
        return 30 - valor
    elif faixa == ' 40 minutos': 
        return 40 - valor
    elif faixa == ' 60 minutos' or faixa == ' Mais que 1 hora': 
        return 60 - valor
df['Tela 77_part_2'] = df.apply(lambda row: classify_time(row['Tela 77_part_2'], row['Tela 77_part_1']), axis=1)    

# Tela 32_part_2 -> Cidade
# Tela 32_part_3 -> Estado

df = df.drop(columns=['Tela 32_part_2', 'Tela 32_part_3', 'Tela 49_part_3', 'Tela 74_part_2', 'Tela 75_part_2', 'Tela 01_part_2','Tela 02_part_3','Tela 77_part_1'])

# Usando ExcelWriter para adicionar a nova aba ao arquivo Excel existente
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='TDados', index=False)